In [64]:
import pandas as pd
from fast_autocomplete import AutoComplete
from typing import Dict, Tuple

In [65]:
## helper functions
def load_searchInfo(file_info: str):
    df = pd.read_csv(file_info, sep='\t', index_col=0)
    words = {}
    for key, row in df.iterrows():
        words.update({key: {'key':key}})
        words.update({row['gene']: {'key':key}})
        words.update({row['product']: {'key':key}})
        
        # skip empty note
        if row['note'] == '<note>':
            continue
            
        for note in str(row['note']).split(';'):
            words.update({note: {'key':key}})
    return df, words    

In [63]:
# read search terms
file_info = '../data/tableInfo.tsv'
#df = pd.read_csv(file_info, sep='\t', index_col=0)
df, words = load_searchInfo(file_info)
print(df['Camk2a'])
print(words.loc['P63323'])
# for key, row in df.iterrows():
#     print(key)
#     break

{'key': 'P11798'}


AttributeError: 'dict' object has no attribute 'loc'

In [11]:
## interact with JSON dictionary
def autocomplete_export(words: Dict[str, Dict[str, str]], file_json: str):
    with open(file_json, "w") as fp:
        json.dump(words , fp)


def autocomplete_load(file_json: str) -> Dict[str, Dict[str, str]]:
    with open(file_json, 'r') as fp:
        words = json.load(fp)
    return words


autocomplete_export(words, '../data/autocomplete_info.json')
test = autocomplete_load('../data/autocomplete_info.json')
print(test['Camk2a'])

{'protein': 'P11798', 'gene': 'Camk2a', 'count': 0}


In [29]:
autocomplete = AutoComplete(words=words)
autocomplete.search(word='Camk', max_cost=3, size=10)
print(autocomplete.words['Camk2a'])
print(autocomplete.get_count_of_word('Camk2a'))
autocomplete.words['Camk2a']['count'] += 1
autocomplete.update_count_of_word(word='Camk2a', count=autocomplete.words['Camk2a']['count'])
print(autocomplete.get_count_of_word('Camk2a'))
print(autocomplete.words['Camk2a'])

{'protein': 'P11798', 'gene': 'Camk2a', 'count': 0}
0
1
{'protein': 'P11798', 'gene': 'Camk2a', 'count': 1}


In [34]:
# test sorting
query = autocomplete.search(word='CALM', max_cost=3, size=10)
list = []
for sublist in query:
    list.append(sublist[-1])
#query = [item for sublist in query for item in sublist]
print(list)
#autocomplete_export(autocomplete.words, '../data/autocomplete_info_test.json')

['CALM', 'Calml3', 'Calmegin', 'Calmodulin-like protein 3', 'Calmodulin-binding protein P-57', 'Calmodulin regulator protein PCP4', 'Calmodulin-dependent calcineurin A subunit beta isoform', 'Calmodulin-regulated spectrin-associated protein 3']
